## Project Start


In [ ]:
from tqdm import tqdm

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os

#from skimage.feature import hog
#from skimage import color, exposure
# images are divided up into vehicles and non-vehicles


def get_image_list(path):
    images_folder = path
    image_array = []
    image_folders = os.listdir(images_folder)
    #print(image_folders)   
    for folder in image_folders :
        image_array.extend(glob.glob(path+folder+'/*'))
        #print ( path+folder)
    print ("Images : " ,len(image_array))
    return image_array
        
# Define a function to return some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    example_img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = example_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = example_img.dtype
    # Return data_dict
    return data_dict

In [ ]:
def print_random_car_notcar(cars, notcars):
    # Just for fun choose random car / not-car indices and plot example images   
    car_ind = np.random.randint(0, len(cars))
    notcar_ind = np.random.randint(0, len(notcars))
    # Read in car / not-car images
    car_image = mpimg.imread(cars[car_ind])
    notcar_image = mpimg.imread(notcars[notcar_ind])
    # Plot the examples
    fig = plt.figure()
    plt.subplot(121)
    plt.imshow(car_image)
    plt.title('Example Car Image')
    plt.subplot(122)
    plt.imshow(notcar_image)
    plt.title('Example Not-car Image')

In [ ]:
from skimage.feature import hog


def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False,
                     feature_vec=True):                        

    return_list = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise= vis, feature_vector= feature_vec)
    if vis:
        hog_features = return_list[0]
        hog_image = return_list[1]
        return hog_features, hog_image
    else:
        hog_features = return_list
        return hog_features

In [ ]:
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in tqdm(imgs):
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features



print('Done')

In [ ]:
def combine_dataset(features_car, features_notcar):
    dataset = np.vstack((features_car, features_notcar)).astype(np.float64)
    dataset_labels = np.hstack(( np.ones(len(features_car)), np.zeros(len(features_notcar))))
    print( len(dataset))
    return dataset, dataset_labels


In [ ]:
import os.path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC






def get_X_train_Y_train(cars, notcars):
    print("cars :", len(cars))
    print("notcars :", len(notcars))
    print('Pickle not found')
    
    colorspace = 'YCrCb'
    orient = 9
    pix_per_cell = 8
    cell_per_block = 3
    hog_channel = 'ALL'

    features_car = extract_features(cars, cspace=colorspace, orient=orient, 
                            pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                            hog_channel=hog_channel)
    features_notcar = extract_features(notcars, cspace=colorspace, orient=orient, 
                            pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                            hog_channel=hog_channel)

    print("hog_images_car :", len(features_car))
    print("hog_images_notcar :", len(features_notcar))
    print(features_car[0].shape)
    print(features_notcar[0].shape)

    dataset, dataset_labels = combine_dataset(features_car, features_notcar)

    print(len(dataset), "    ", len(dataset_labels))
    print(len(features_car), "   ", len(features_notcar))
    print(np.ones_like(1196))
    X_train , X_test, y_train, y_test = train_test_split(dataset, dataset_labels, test_size = 0.2)

    return X_train, X_test, y_train, y_test

In [ ]:
def train_svm(X_train , X_test, y_train, y_test):
    
    clf = SVC(kernel = 'linear')
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    
    print(acc)
        
    return clf, acc




In [ ]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features



In [ ]:
def find_cars(img, ystarts, ystops, scales, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    bboxes = []
    
    for scale, ystart, ystop in zip(scales, ystarts, ystops) :
        img_tosearch = img[ystart:ystop,:,:]
        ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]

        # Define blocks and steps as above
        nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
        nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
        nfeat_per_block = orient*cell_per_block**2

        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1

        # Compute individual channel HOG features for the entire image
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

        for xb in range(nxsteps):
            for yb in range(nysteps):
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                # Extract HOG for this patch
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

                xleft = xpos*pix_per_cell
                ytop = ypos*pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

                # Get color features
                #spatial_features = bin_spatial(subimg, size=spatial_size)
                #hist_features = color_hist(subimg, nbins=hist_bins)

                # Scale features and make a prediction
                #test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
                
                
                test_features = X_scaler.transform(np.array(hog_features).reshape(1, -1))
                
                #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
                test_prediction = svc.predict(test_features)

                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    bboxes.append([(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)])
                    cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)
#     plt.figure()
#     plt.imshow(draw_img)

    return bboxes, draw_img

In [ ]:
import pickle

def main_function():

    #cars = get_image_list('training_data/vehicles_smallset/')
    #notcars = get_image_list('training_data/non-vehicles_smallset/')
#     cars = get_image_list('training_data/vehicles/')
#     notcars = get_image_list('training_data/non-vehicles/')
    
#     print('All images loaded')
    
#     data_info = data_look(cars, notcars)

#     print('Your function returned a count of', 
#           data_info["n_cars"], ' cars and', 
#           data_info["n_notcars"], ' non-cars')
#     print('of size: ',data_info["image_shape"], ' and data type:', 
#           data_info["data_type"])

#     print_random_car_notcar(cars, notcars)

#     random_hog(cars)

    check_if_exits = os.path.isfile("pickle_data.p") 

    if check_if_exits == True :
        read_pickle_data = pickle.load( open( "pickle_data.p", "rb" ) )
        X_scaler = read_pickle_data["X_scaler"]
        svc = read_pickle_data["svc"]
        acc = read_pickle_data["acc"]
        print('Data read from pickle')
        print(X_scaler)
        print("Accuracy : ", acc)
    else :
        print('Error reading pickle')

        X_train, X_test, y_train, y_test = get_X_train_Y_train(cars, notcars)
        
        print ("cars_X_train : ", len(X_train))
        print ("cars_X_test : ", len(X_test))

        X_scaler = StandardScaler().fit(X_train)
        # Apply the scaler to X
        X_train = X_scaler.transform(X_train)
        X_test = X_scaler.transform(X_test)
        
        
        print(len(X_train[0]))
        
        svc, acc = train_svm(X_train , X_test, y_train, y_test)

        write_pickle_data = {}
        write_pickle_data["X_scaler"] = X_scaler
        write_pickle_data["svc"] = svc     
        write_pickle_data['acc'] = acc
        pickle.dump( write_pickle_data, open( "pickle_data.p", "wb" ) )
    return svc, X_scaler

In [ ]:
svc, X_scaler = main_function()

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes

def remove_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] -= 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [ ]:
from scipy.ndimage.measurements import label
from queue import Queue

img = mpimg.imread('test_images/test1.jpg')
box_queue = Queue(maxsize=6)
heat = np.zeros_like(img[:,:,0]).astype(np.float)

svc, X_scaler = main_function()

def process_image(frame):
    
    global heat 
    global box_queue
    global count
    global img_save

    #if count
    ystart = [400,400,400,400,464]
    ystop = [496,528,560,592,656]
    #ystart = 400
    #ystop = 656
    scale = [1,1.5,2,3,4] #4 #1.5 
    color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 9  # HOG orientations
    pix_per_cell = 8 # HOG pixels per cell
    cell_per_block = 3 # HOG cells per block
    hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    spatial_size = (16, 16) # Spatial binning dimensions
    hist_bins = 16    # Number of histogram bins
    spatial_feat = False # Spatial features on or off
    hist_feat = False # Histogram features on or off
    hog_feat = True # HOG features on or off

    if box_queue.full():
        heat = remove_heat(heat,box_queue.get())

    bboxes, box_img = find_cars(frame, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)

    bboxes.append(bboxes)
    bboxes.append(bboxes)
    bboxes.append(bboxes)
    
    box_queue.put(bboxes)
    

    # Add heat to each box in box list
    heat = add_heat(heat,bboxes)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(frame), labels)

    heatmap_image = (np.dstack((heatmap, heatmap, heatmap))*255).astype(np.uint8)
    
    stack1 = np.hstack((heatmap_image, box_img))
    stack2 = np.hstack((draw_img, draw_img))
    #print (stack1.shape)
    #print(draw_img.shape)
    
    stack3 = np.vstack((stack1, stack2))
    return stack3
    
    
    
    

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'test_videos_output/test_full_2.mp4'


count = 0
img_save = None
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #.subclip(2,4) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))